### Methods proposed in the article 

In [1]:
import numpy as np
import os
import sys
project_dir = os.getcwd().split('notebooks')[0]
sys.path.append(project_dir)

In [2]:
from src.helper import *

#### 1) Algorithm 1: NaiveMapper

In [17]:
def NaiveMapper(A):
    """
    Function that computes sparse matrix multiplication
    input: 
    A: sparse matrix
    output:
    AProd: dot products between columns of A
    """
    (n,m) = A.shape
    AProd = sum( [A[i][0]*A[i] for i in range(len(A))] ) # we walk through the rows
    for r in range(1,m): # then emit the columns
        AProd = np.vstack((AProd,sum( [A[i][r]*A[i] for i in range(len(A))] ) ))
    return AProd
    

#### 2) Algorithm 2: NaiveReducer

In [22]:
def NaiveReducer(C):
    """
    Function that computes the sum of lists of arrays
    input:
    C: list of arrays
    output:
    sum(C): sum of list of arrays
    """
    return sum(C)

In [23]:
# Example
A = sparse_generator(3,4)
NaiveMapper(A)

#A^T A:
NaiveReducer(NaiveMapper(A))

array([0.50155941, 0.21055472, 0.        , 0.11044557])

#### 3) Algorithm 3: DIMSUMMapper

In [25]:
A

array([[ 0.        , -0.45886242,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.33233352],
       [ 0.70820859,  0.        ,  0.        ,  0.        ]])

In [ ]:
def DIMSUMMApper(A, gamma):
    (n,m) = A.shape
    for i in range(n):